# 自然言語処理: LDA

参考：
* [Gensim](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Corpora_and_Vector_Spaces.ipynb)
* [Gensim preprocessing & training LDA](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/lda_training_tips.ipynb)
* [NLTK]()
* [Sam Roweis' website](http://www.cs.nyu.edu/~roweis/data.html) : Download texts of NIPS papers

## Libraries

In [1]:
import os
from nltk.tokenize import regexp_tokenize
from gensim import corpora
from smart_open import smart_open
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

## Example data

In [2]:
# Folder containing all NIPS papers.
data_dir = './nipstxt/'

# Folders containin individual NIPS papers.
yrs = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
dirs = ['nips' + yr for yr in yrs]

# Read all texts into a list.
docs = []
for yr_dir in dirs:
    files = os.listdir(data_dir + yr_dir)
    for filen in files:
        # Note: ignoring characters that cause encoding errors.
        with smart_open(data_dir + yr_dir + '/' + filen, 'rb') as fid:
            txt = str(fid.read()).replace("\\n", "")
        docs.append(txt)

/Users/maruyamatooru/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Create dictionary & corpus

In [3]:
# tokenize the documents
tokens = [regexp_tokenize(d.lower(), r"\w+") for d in docs] # using NLTK library

# Remove numbers, but not words that contain numbers.
tokens = [[t for t in token if not t.isnumeric()] for token in tokens]

# Remove words that are only one character.
tokens = [[t for t in token if len(t) > 1] for token in tokens]

tokens[0][:10]

['connectivity',
 'versus',
 'entropy',
 'yaser',
 'abu',
 'mostafa',
 'california',
 'institute',
 'of',
 'technology']

In [4]:
# Create dictionary with the tokens
dictionary = corpora.Dictionary(tokens)
# dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)

Dictionary(83764 unique tokens: ['0a', '2h', '2h2', '2he', '2n']...)


In [5]:
# filtering of tokens
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)
print(dictionary)

Dictionary(7412 unique tokens: ['2n', '_c', 'a2', 'ability', 'abu']...)


In [6]:
# how to use
print(dictionary.token2id["2n"]) # token=2nのid
print(dictionary.dfs[0]) # id=0 (token="2n")が現れた回数

0
97


In [7]:
# コーパス
corpus = [dictionary.doc2bow(t) for t in tokens]
corpus[0][:10]

[(0, 4),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 4),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [8]:
# 新しい文書を辞書で
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
new_vec

[(169, 1), (1929, 1)]

## Save&Load corpus

In [9]:
# Save corpus as efficient file formats 
corpora.MmCorpus.serialize(os.path.join("./", 'corpus.mm'), corpus) # MatrixMarket (mtx) format
# corpora.SvmLightCorpus.serialize(os.path.join("./", 'corpus.svmlight'), corpus) # SVM light

/Users/maruyamatooru/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# Load corpus
corpus = corpora.MmCorpus(os.path.join("./", 'corpus.mm'))
corpus

/Users/maruyamatooru/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
corpus = list(corpus)
corpus[0][:10]

/Users/maruyamatooru/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[(0, 4.0),
 (1, 1.0),
 (2, 1.0),
 (3, 2.0),
 (4, 4.0),
 (5, 1.0),
 (6, 1.0),
 (7, 1.0),
 (8, 1.0),
 (9, 1.0)]

## gensim: LDA

In [12]:
from gensim.models import LdaModel

* n_topics: トピック数
* chunksize: 学習の1ステップで何個のDocumentを使うか。学習の結果に強く影響する（[ref](https://www.di.ens.fr/~fbach/mdhnips2010.pdf)）
* passes: エポック数。各documentが何回学習に用いられるか。
* iterations
* eval_every: 
* alpha, eta="auto": 最適なハイパーパラメータを自動で探索する
* coherence: トピックの解釈性の高さを表す指標([ref](https://www.slideshare.net/hoxo_m/coherence-57598192))。いくつかの手法が実装されており選択できる([ref](https://qiita.com/tatsuya-miyamoto/items/7d49959c74f3c1e0cf63))。u_mass以外は学習用データとは異なるテキストが必要。RWDにはu_mass以外選択の余地がなさそう。

In [15]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.
alpha='auto'
eta='auto'

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus[:100], id2word=id2word, \
                       chunksize=chunksize, alpha=alpha, eta=eta, \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every) 

CPU times: user 6.84 s, sys: 34.6 ms, total: 6.87 s
Wall time: 3.49 s


In [16]:
coherence="u_mass"
top_topics = model.top_topics(corpus[:100], topn=20, coherence=coherence)
top_topics[0]

([(0.011323489, 'memory'),
  (0.0060359915, 'matrix'),
  (0.005187462, 'net'),
  (0.0051225144, 'vectors'),
  (0.0048700874, 'capacity'),
  (0.004830101, 'associative'),
  (0.0046276394, 'units'),
  (0.0045003598, 'stored'),
  (0.004232521, 'patterns'),
  (0.00415348, 'fig'),
  (0.0040757237, 'hopfield'),
  (0.003869393, 'layer'),
  (0.0034759135, 'equation'),
  (0.0034571607, 'sequence'),
  (0.0034305546, 'recall'),
  (0.0031486442, 'image'),
  (0.0031168624, 'control'),
  (0.0030738767, 'xf8'),
  (0.0029989926, 'optical'),
  (0.0027731087, 'signal')],
 -0.8729128881423159)

In [17]:
# Average topic coherence
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

Average topic coherence: -1.3912.


In [18]:
pyLDAvis.gensim.prepare(model, corpus[:100], dictionary)

/Users/maruyamatooru/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.130567  0.002589       1        1  16.911903
6     -0.008822 -0.004829       2        1  15.657844
5     -0.025979 -0.008678       3        1  13.067381
4     -0.042921 -0.052501       4        1  12.585778
0      0.001544 -0.070114       5        1  12.421408
7     -0.036190  0.057595       6        1   8.296198
2     -0.048977  0.032113       7        1   6.899766
8      0.004760 -0.023602       8        1   5.989604
9     -0.017597  0.031409       9        1   4.320777
3      0.043616  0.036018      10        1   3.849346, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
1064  Default  618.000000           units  618.000000  30.0000  30.0000
686   Default  313.000000          hidden  313.000000  29.0000  29.0000
3222  Default  125.000000      classifier  125.000000  28.0000  28.0000
802   Default  332.000000             net  332.000000  27.0000  27.0000
55    Default  374.000000           cells  374.000000  26.0000  26.0000
1063  Default  369.000000            unit  369.000000  25.0000  25.0000
2366  Default  322.000000            cell  322.000000  24.0000  24.0000
774   Default  452.000000          memory  452.000000  23.0000  23.0000
739   Default  488.000000           layer  488.000000  22.0000  22.0000
2852  Default  204.000000          firing  204.000000  21.0000  21.0000
1843  Default  134.000000          cortex  134.000000  20.0000  20.0000
4271  Default   64.000000         visible   64.000000  19.0000  19.0000
1089  Default  304.000000          weight  304.000000  18.0000  18.0000
4312  Default   80.000000           joint   80.000000  17.0000  17.0000
1549  Default  201.000000           image  201.000000  16.0000  16.0000
899   Default  100.000000         regions  100.000000  15.0000  15.0000
357   Default  120.000000          visual  120.000000  14.0000  14.0000
906   Default  200.000000  representation  200.000000  13.0000  13.0000
2909  Default   69.000000         message   69.000000  12.0000  12.0000
1803  Default  199.000000        capacity  199.000000  11.0000  11.0000
1815  Default   59.000000              cn   59.000000  10.0000  10.0000
1616  Default  197.000000            node  197.000000   9.0000   9.0000
2992  Default   94.000000           spike   94.000000   8.0000   8.0000
1266  Default   55.000000           match   55.000000   7.0000   7.0000
2387  Default  118.000000        cortical  118.000000   6.0000   6.0000
3430  Default   63.000000          fibers   63.000000   5.0000   5.0000
3392  Default  165.000000     convergence  165.000000   4.0000   4.0000
217   Default  535.000000         neurons  535.000000   3.0000   3.0000
923   Default  105.000000            role  105.000000   2.0000   2.0000
870   Default  174.000000     propagation  174.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
3607  Topic10    9.193460        messages   25.270550   2.2461  -6.2817
4021  Topic10    3.424384           score    7.267569   2.5048  -7.2693
3864  Topic10    9.038612           nerve   26.774242   2.1713  -6.2987
3430  Topic10   15.714929          fibers   63.330296   1.8635  -5.7456
2224  Topic10   10.161137          expert   34.910400   2.0231  -6.1816
4597  Topic10    8.601460   environmental   28.315334   2.0658  -6.3483
1272  Topic10   13.292052            move   55.846840   1.8218  -5.9131
3601  Topic10   10.353175           links   38.716736   1.9383  -6.1629
1616  Topic10   26.046955            node  197.385941   1.2320  -5.2403
745   Topic10    5.806388         leaming   16.264908   2.2272  -6.7413
662   Topic10   13.293537            gain   71.353065   1.5769  -5.9129
994   Topic10   13.990995        strength   80.022224   1.5134  -5.8618
640   Topic10   15.876451        features  112.027054   1.3034  -5.73